<a href="https://colab.research.google.com/github/gnoparus/bualabs/blob/master/nbs/31d_chatgpt_prompt_check_precondition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ต่อไปเราจะมาเรียนรู้ Prompt Engineering เทคนิคที่ 3 สั่งให้ AI ตรวจเช็คเงื่อนไขที่จำเป็น ก่อนทำงาน

# 0. Magic Commands
ให้ใส่ไว้บนสุดทุก Notebook เป็นการสั่งให้ Notebook ก่อนรัน ให้รีโหลด Library ภายนอกที่เรา import ไว้ใหม่โดยอัตโนมัติ

และให้พล็อตกราฟ matplotlib ใน Output ของ cell แบบ code ได้เลย

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

สั่งให้ Colab แสดงผลข้อความแบบ Wrap Text

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# 1. Import Library

ติดตั้ง OpenAI Library ด้วยการ Uncomment Cell ด้านล่างแล้ว Run

In [1]:
# ! pip install openai

In [ ]:
import openai

# 2. Authentication

ก่อนที่เราจะเรียกใช้งาน ChatGPT, GPT-4 ของ OpenAI ได้ เราต้องลงทะเบียน สมัครใช้งาน และใส่ข้อมูลวิธีชำระเงินให้เรียบร้อยก่อน 

## Sign up

กดปุ่ม Sign up สีเขียว ในลิงค์ด้านล่าง สมัครใช้งาน OpenAI 

https://platform.openai.com/overview

ใส่ข้อมูลบัตรเครดิต ในหน้า Billing ให้เรียบร้อย

https://platform.openai.com/account/billing/payment-methods






## API Key

สร้าง API Key จากลิงค์ด้านล่าง โดยกดปุ่ม Create new secret key ตั้งชื่อ แล้ว Copy secret key เตรียมไว้ใส่ในช่อง OPENAI_API_KEY ด้านล่าง

https://platform.openai.com/account/api-keys


## ตัวอย่าง API Key 

OPENAI_API_KEY: sk-abcDeFGHijKlMNopQrstuvWxyZ1234567890abcDefhiJklm

รัน Cell ด้านล่าง แล้วใส่ secret key ที่ได้มา

In [ ]:
import os
from getpass import getpass

token = getpass('Enter OPENAI_API_KEY here: ')
# print ('token is', token)

openai.api_key = token

Enter OPENAI_API_KEY here: ··········


หลังจากทดสอบเสร็จแล้ว แนะนำให้กลับไปลบ Secret Key ในหน้า https://platform.openai.com/account/api-keys ทิ้งไปทุกครั้ง เพื่อความปลอดภัย โดยเราสามารถสร้าง Key ใหม่ได้เรื่อย ๆ ไม่จำกัด

# 3. ฟังก์ชัน

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [ ]:
resp = get_completion("what are the 5 most populated provinces in thailand?")

In [ ]:
print(resp)

1. Bangkok - population of approximately 8.3 million
2. Nonthaburi - population of approximately 1.2 million
3. Samut Prakan - population of approximately 1.1 million
4. Chonburi - population of approximately 1.1 million
5. Ubon Ratchathani - population of approximately 1.1 million


# 4. ตัวอย่างการเขียน Prompt ไม่ตรวจเช็คเงื่อนไขที่จำเป็น ก่อนทำงาน

เราจะลองสั่งให้โมเดล ลิสต์รายชื่อทีมฟุตบอล 10 ทีม ที่อยู่ในข้อความข่าว ที่คั่นด้วย เครื่องหมาย ` 

แต่ในข้อความข่าว มีชื่อทีมอยู่ไม่ถึง 10 ชื่อ โมเดลจะทำอย่างไร

In [ ]:
prompt = """
List the names of 10 soccer teams mentioned in the text. The text is delimited by triple backticks.

```
Real Madrid are in advanced talks with Borussia Dortmund to sign England midfielder Jude Bellingham for more than 100m euros.

The 19-year-old, who joined Dortmund from Birmingham City in July 2020, was one of England's top performers at last year's World Cup.

Real are willing to pay around 100m euros for Bellingham but Dortmund are understood to want 150m euros.

Bellingham has been linked with some of Europe's top sides.

Real are confident the midfielder - who has turned down a contract offer to stay at Dortmund - wants to join them but Manchester City are still interested too.

Bellingham has played 40 times for his club this season - scoring 11 goals and registering seven assists.
```
"""

resp = get_completion(prompt)

In [ ]:
print(resp)

1. Real Madrid
2. Borussia Dortmund
3. Birmingham City
4. England
5. World Cup
6. Europe
7. Manchester City
8. Dortmund
9. midfielder
10. club


โมเดลก็พยายามอย่างที่สุด ที่จะปฏิบัติตามคำสั่ง ถึงแม้จะเป็นไปไม่ได้ ก็เอาจนได้ โดยสร้างชื่อทีมที่ไม่มีอยู่จริง ในข้อความข่าวขึ้นมา หยิบมาจากไหนก็ไม่รู้ เพื่อให้ครบ 10 ทีม

# 5. ตัวอย่างการเขียน Prompt ที่ตรวจเช็คเงื่อนไขที่จำเป็น ก่อนทำงาน

ในเคสนี้ เราจะสั่งให้โมเดลเช็คว่า ชื่อทีมฟุตบอล มีไม่ถึง 10 ชื่อ ก็ไม่เป็นไร เอาเท่าที่มีก็ได้

In [ ]:
prompt2 = """
List the names of 10 teams mentioned in the text. The text is delimited by triple backticks.
Check if there are fewer than 10 teams, just respond with what we have.

```
Real Madrid are in advanced talks with Borussia Dortmund to sign England midfielder Jude Bellingham for more than 100m euros.

The 19-year-old, who joined Dortmund from Birmingham City in July 2020, was one of England's top performers at last year's World Cup.

Real are willing to pay around 100m euros for Bellingham but Dortmund are understood to want 150m euros.

Bellingham has been linked with some of Europe's top sides.

Real are confident the midfielder - who has turned down a contract offer to stay at Dortmund - wants to join them but Manchester City are still interested too.

Bellingham has played 40 times for his club this season - scoring 11 goals and registering seven assists.
```
"""

resp2 = get_completion(prompt2)

In [ ]:
print(resp2)

1. Real Madrid
2. Borussia Dortmund
3. Birmingham City
4. England national team
5. Manchester City
6. Europe's top sides (not a specific team)
7.


โมเดลก็จะทำงานได้อย่างถูกต้อง ตอบจาก 6-7 ชื่อเท่าที่มีอยู่ใน ข้อความข่าว

# 6. สรุป

- ถ้าเราไม่สั่งให้โมเดลเช็คเงื่อนไขก่อนทำงาน คำตอบที่ได้จะออกมาผิด เนื่องจากเป็นไปไม่ได้ แต่โมเดลก็ยังพยายามตอบออกมา
- เมื่อเราสั่งให้โมเดลเช็คก่อน ผลลัพท์ที่ได้ออกมาดีขึ้น ถูกต้องมากขึ้น